# Project 2: Supervised Learning
### Building a Student Intervention System

## 1. Classification vs Regression

Your goal is to identify students who might need early intervention - which type of supervised machine learning problem is this, classification or regression? Why?

Classification, because we're classifying students as either "on pace" or not. The output will be a label as opposed to a number.

## 2. Exploring the Data

Let's go ahead and read in the student dataset first.

_To execute a code cell, click inside it and press **Shift+Enter**._

In [2]:
# Import libraries
import numpy as np
import pandas as pd

In [3]:
# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"
# Note: The last column 'passed' is the target/label, all other are feature columns

Student data read successfully!


Now, can you find out the following facts about the dataset?
- Total number of students
- Number of students who passed
- Number of students who failed
- Graduation rate of the class (%)
- Number of features

_Use the code block below to compute these values. Instructions/steps are marked using **TODO**s._

In [4]:
# TODO: Compute desired values - replace each '?' with an appropriate expression/function call
from pandas import Series

n_students, n_features = student_data.shape
n_passed = student_data[student_data.passed == 'yes'].count().size
n_failed = student_data[student_data.passed == 'no'].count().size
grad_rate = 0
print "Total number of students: {}".format(n_students)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Number of features: {}".format(n_features)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of students who passed: 31
Number of students who failed: 31
Number of features: 31
Graduation rate of the class: 0.00%


## 3. Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Let's first separate our data into feature and target columns, and see if any features are non-numeric.<br/>
**Note**: For this dataset, the last column (`'passed'`) is the target or label we are trying to predict.

In [5]:
# Extract feature (X) and target (y) columns
feature_cols = list(student_data.columns[:-1])  # all columns but last are features
target_col = student_data.columns[-1]  # last column is the target/label
print "Feature column(s):-\n{}".format(feature_cols)
print "Target column: {}".format(target_col)

X_all = student_data[feature_cols]  # feature values for all students
y_all = student_data[target_col]  # corresponding targets/labels
print "\nFeature values:-"
print X_all.head()  # print the first 5 rows

Feature column(s):-
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']
Target column: passed

Feature values:-
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...    

### Preprocess feature columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation.

In [6]:
# Preprocess feature columns
def preprocess_features(X):
    outX = pd.DataFrame(index=X.index)  # output dataframe, initially empty

    # Check each column
    for col, col_data in X.iteritems():
        # If data type is non-numeric, try to replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])
        # Note: This should change the data type for yes/no columns to int

        # If still non-numeric, convert to one or more dummy variables
        if col_data.dtype == object:
            col_data = pd.get_dummies(col_data, prefix=col)  # e.g. 'school' => 'school_GP', 'school_MS'

        outX = outX.join(col_data)  # collect column(s) in output dataframe

    return outX

X_all = preprocess_features(X_all)
print "Processed feature columns ({}):-\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48):-
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Split data into training and test sets

So far, we have converted all _categorical_ features into numeric values. In this next step, we split the data (both features and corresponding labels) into training and test sets.

In [75]:
# First, decide how many training vs test samples you want
num_all = student_data.shape[0]  # same as len(student_data)
num_train = 300  # about 75% of the data
num_test = num_all - num_train

# TODO: Then, select features (X) and corresponding labels (y) for the training and test sets
# Note: Shuffle the data or randomly select samples to avoid any bias due to ordering in the dataset
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=300)
print "Training set: {} samples".format(X_train.shape[0])
print "Test set: {} samples".format(X_test.shape[0])
# Note: If you need a validation set, extract it from within training data

Training set: 300 samples
Test set: 95 samples


## 4. Training and Evaluating Models
Choose 3 supervised learning models that are available in scikit-learn, and appropriate for this problem. For each model:

**What is the theoretical O(n) time & space complexity in terms of input size?**

Nearest Neighbors: Time complexity is O(N^2) using brute force or O(NlogN) using the KD Tree algorithm.

Decision Trees: Time complexity is O(logn).

Support Vector Machines: Time complexity is between O(N^2) and O(N^3). 

Neural Networks: "Suppose there are n training samples, m features, k hidden layers, each containing h neurons - for simplicity, and o output neurons. The time complexity of backpropagation is O(n*m*h^k*o*i), where i is the number of iterations."

**What are the general applications of this model? What are its strengths and weaknesses?**

Nearest Neighbors: 
Neural Networks: Useful for image recognition (e.g recognizing digits in the MNIST dataset) but generally slowest to train of the other model.


**Given what you know about the data so far, why did you choose this model to apply?**

Neural Networks: While NNs are more often used for image recognition, they're still able to universially approximate any function so I'm curious to see how they perform here.

**Fit this model to the training data, try to predict labels (for both training and test sets), and measure the F<sub>1</sub> score. Repeat this process with different training set sizes (100, 200, 300), keeping test set constant.**

Produce a table showing training time, prediction time, F<sub>1</sub> score on training set and F<sub>1</sub> score on test set, for each training set size.

Note: You need to produce 3 such tables - one for each model.

In [61]:
# Train a model
import time

def train_classifier(clf, X_train, y_train):
    print "Training {}...".format(clf.__class__.__name__)
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print "Training time (secs): {:.3f}".format(end - start)

# TODO: Choose a model, import it and instantiate an object
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier()

# Fit model to training data
train_classifier(clf, X_train, y_train)  # note: using entire training set here
#print clf  # you can inspect the learned model by printing it

Training KNeighborsClassifier...
Training time (secs): 0.001


In [66]:
# Predict on training set and compute F1 score
from sklearn.metrics import accuracy_score, precision_score, recall_score

def predict_labels(clf, features, target):
    print "Predicting labels using {}...".format(clf.__class__.__name__)
    start = time.time()
    y_pred = clf.predict(features)
    end = time.time()
    print "Prediction time (secs): {:.3f}".format(end - start)
    return accuracy_score(target.values, y_pred), precision_score(target.values, y_pred, pos_label='yes'), recall_score(target.values, y_pred, pos_label='yes')

train_f1_score = predict_labels(clf, X_train, y_train)
print "F1 score for training set: {}".format(train_f1_score)

Predicting labels using RandomForestClassifier...
Prediction time (secs): 0.008
F1 score for training set: (0.68999999999999995, 0.68999999999999995, 1.0)


In [67]:
# Predict on test data
print "F1 score for test set: {}".format(predict_labels(clf, X_test, y_test))

Predicting labels using RandomForestClassifier...
Prediction time (secs): 0.007
F1 score for test set: (0.61052631578947369, 0.61052631578947369, 1.0)


In [68]:
# Train and predict using different training set sizes
def train_predict(clf, X_train, y_train, X_test, y_test):
    print "------------------------------------------"
    train_classifier(clf, X_train, y_train)
    print "Accuracy, precision, and recall scores for training set: \n {}".format(predict_labels(clf, X_train, y_train))
    print "Accuracy, precision, and recall scores for test set:\n {}".format(predict_labels(clf, X_test, y_test))

# TODO: Run the helper function above for desired subsets of training data
# Note: Keep the test set constant

In [74]:
clf = KNeighborsClassifier(n_neighbors=5)
train_predict(clf, X_train, y_train, X_test, y_test)

from sklearn import tree
clf = tree.DecisionTreeClassifier(max_depth=1)
train_predict(clf, X_train, y_train, X_test, y_test)

from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(2, 2), random_state=1)
train_predict(clf, X_train, y_train, X_test, y_test)

from sklearn import svm
clf = svm.SVC()
train_predict(clf, X_train, y_train, X_test, y_test)

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(max_depth=1)
train_predict(clf, X_train, y_train, X_test, y_test)

------------------------------------------
Training KNeighborsClassifier...
Training time (secs): 0.001
Predicting labels using KNeighborsClassifier...
Prediction time (secs): 0.006
Accuracy, precision, and recall scores for training set: 
 (0.79000000000000004, 0.7975206611570248, 0.93236714975845414)
Predicting labels using KNeighborsClassifier...
Prediction time (secs): 0.005
Accuracy, precision, and recall scores for test set:
 (0.71578947368421053, 0.70666666666666667, 0.91379310344827591)
------------------------------------------
Training DecisionTreeClassifier...
Training time (secs): 0.001
Predicting labels using DecisionTreeClassifier...
Prediction time (secs): 0.000
Accuracy, precision, and recall scores for training set: 
 (0.71333333333333337, 0.7531380753138075, 0.86956521739130432)
Predicting labels using DecisionTreeClassifier...
Prediction time (secs): 0.000
Accuracy, precision, and recall scores for test set:
 (0.75789473684210529, 0.73972602739726023, 0.9310344827586

## 5. Choosing the Best Model

**Based on the experiments you performed earlier, in 1-2 paragraphs explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?**

The data set is small enough that resources aren't a bottle neck for any of the models. Of the models tested, Decision Trees performed the best with the max depth set to 1. This seems to imply that the best prediction model is a simple one, and that only a few (maybe only one) of the features are predictive of student grad rates.

**In 1-2 paragraphs explain to the board of supervisors in layman's terms how the final model chosen is supposed to work (for example if you chose a Decision Tree or Support Vector Machine, how does it make a prediction).**

The Decision Tree iterates over the features and looking for whichever one splits the data best, then creates a "tree" that splits the data on that feature and continues this process for the remaining set of features.

**Fine-tune the model. Use Gridsearch with at least one important parameter tuned and with at least 3 settings. Use the entire training set for this.**

**What is the model's final F<sub>1</sub> score?**

In [93]:
from sklearn.model_selection import GridSearchCV

decision_tree = tree.DecisionTreeClassifier(max_depth=1)
clf = GridSearchCV(estimator=decision_tree, param_grid=dict(max_depth=range(1,10)))
clf.fit(X_all, y_all)

print clf.cv_results_['rank_test_score']

from sklearn.metrics import f1_score
clf = tree.DecisionTreeClassifier(max_depth=1)
train_classifier(clf, X_train, y_train)
y_pred = clf.predict(X_test)
print f1_score(y_pred, y_test)

[1 2 8 3 4 5 7 6 9]
Training DecisionTreeClassifier...
Training time (secs): 0.001


/usr/local/lib/python2.7/site-packages/sklearn/metrics/classification.py:1008: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if pos_label not in present_labels:


ValueError: pos_label=1 is not a valid label: array(['no', 'yes'], 
      dtype='|S3')